# Interacting with Kinetica vector similarity search via LLM
### Install required packages

In [0]:
!pip install gpudb==7.2.0.0b0 pandas pyarrow typeguard langchain langchain_openai nemollm==0.3.5 colorlog "langchain-kinetica @ git+https://git@github.com/kineticadb/langchain-kinetica.git"

In [ ]:
import kinetica.setup
kinetica.setup.setup()

### Connect to Kinetica and the LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_kinetica import KineticaChatLLM, KineticaSqlOutputParser, SqlResponse

#db = gpudb.GPUdb(host='https://demo72.kinetica.com/_gpudb',
#                 username='gtc',
#                 password='Kinetica123!')

# create the Kinetica connection
kdbc = KineticaChatLLM._create_kdbc(host="https://demo72.kinetica.com/_gpudb", login="gtc", password="Kinetica123!")

# create the Kinetica LLM
kinetica_llm = KineticaChatLLM(kdbc=kdbc)

# Set the context to use
kinetica_ctx = 'nyse.nyse_vector_ctxt'

### Set up the context

In [ ]:
# load the context from the database
ctx_messages = kinetica_llm.load_messages_from_context(kinetica_ctx)

# Add the input prompt. This is where input question will be substituted.
ctx_messages.append(("human", "{input}"))

# Create the prompt template.
prompt_template = ChatPromptTemplate.from_messages(ctx_messages)
prompt_template.pretty_print()

# create the chain. 
# note: The KineticaSqlOutputParser will execute the SQL statement and is optional.
chain = prompt_template | kinetica_llm | KineticaSqlOutputParser(kdbc=kdbc)

### Some questions, including vector similarity search, without the SQL fuss

In [ ]:
from IPython.display import display, HTML

# Here you must ask a question relevant to the LLM context provided in the prompt template.
response: SqlResponse = chain.invoke({"input": '''what stock has traded with the highest volume today?'''})
display(HTML(response.dataframe.to_html(index=False)))
    

In [ ]:
response: SqlResponse = chain.invoke({"input": '''find all sofi stock trades between 2024-01-29 14:25:00 and 2024-01-29 14:35:00 where the price is not null'''})
response.dataframe.plot.line(x='t', y='lp')

In [0]:
response: SqlResponse = chain.invoke({"input": '''find similar patterns to sofi at 2024-01-29 14:25:00.000'''})
display(HTML(response.dataframe.to_html(index=False)))

In [0]:
response: SqlResponse = chain.invoke({"input": '''find all qqq stock trades between 2024-01-22 16:00:00 and 2024-01-22 16:05:00 where the price is not null'''})
response.dataframe.plot.line(x='t', y='lp')

### Get Kinetica LLM and Nemo talking to each other

In [ ]:
import importlib
import kinetica.kineai

importlib.reload(kinetica.kineai)
kineticallm = kinetica.kineai.KineticaLLM('nyse.nyse_vector_ctxt')

In [ ]:
system = """ KineticAI is a cheerful AI assistant for engaging in a conversation between an LLM using the Nemo framework and the Kinetica LLM.  The Kinetica
LLM is designed to translate natural language questions into SQL queries. 

In addition to responding with  natural language it is able to ask questions to a database AI named SqlAssist that can query and summarize the logs. 
If it responds with a "KineticaLLM |  question" where question is sent to the SqlAssist AI. The SqlAssist AI will respond with an answer 
to the question in JSON format to the question made to SqlAssist by KineticAI.

when presented with a question, you should prefix your response with "KineticaLLM | "
if a sentence ends in a "?", you should prefix your response with "KineticaLLM | "

Consider the following example where a user asks KineticAI a question and KineticAI asks a followup question to SqlAssist. KineticAI uses the response from 
SqlAssist to answer the user's question.

user: what is the weather like today?
assistant: KineticaLLM | what is the weather like today?
user: KineticaLLM |  [{"EXPR_0": 5.4}]
assistant: The answer is 5.4
"""

context0 = [dict(role="system", content=system),
            dict(role="user", content="what is the stock price today?"),
            dict(role="assistant", content="KineticaLLM | what is the stock price today?"),
            dict(role="user", content="how many rows of data are you storing?"),
            dict(role="assistant", content="KineticaLLM | how many rows of data are you storing?"),
            dict(role="user", content="what is the average number of telemetry rows per 5 second increment?"),
            dict(role="assistant", content="KineticaLLM | what is the average number of telemetry rows per 5 second increment?"),
            dict(role="user", content="find me top stock prices today"),
            dict(role="assistant", content="KineticaLLM | find me top stock prices today")]

In [ ]:
question = 'what stock symbol other than Apple has the highest price within the last 15 minutes?'
response = kineticallm.chat(context0, question)